## install finrl library
!pip install plotly==4.4.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
!pip install wrds
!pip install swig
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
!pip install PyPortfolioOpt

In [1]:
import os
from finrl import config
from finrl import config_tickers

if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [2]:
from preprocessor.yahoodownloader import YahooDownloader

df = YahooDownloader(start_date = '2008-01-01',
                     end_date = '2021-09-02',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()
print(len(df.tic.unique()))

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (100385, 8)
30


In [3]:
df.head()

Price,date,close,high,low,open,volume,tic,day
0,2008-01-02,5.855758,6.018652,5.786934,5.988898,1079178800,AAPL,2
1,2008-01-02,32.223785,32.528046,31.988676,32.223785,7934400,AMGN,2
2,2008-01-02,38.898682,39.874196,38.708151,39.698909,8053700,AXP,2
3,2008-01-02,63.481640,64.375743,63.027255,64.177869,4303000,BA,2
4,2008-01-02,44.507305,45.792806,44.141823,45.723490,6337800,CAT,2


In [4]:
### Preprocess Data

TO-DO:

1. understand what FeatureEngineer methods are used on this dataframe
2. what technical indicators are used
3. what is turbulence index and how is it calculated
4. data cleaning process

In [5]:
def data_split(df, start, end, target_date_col="date"):
    """
    split the dataset into training or testing using date
    :param data: (df) pandas dataframe, start, end
    :return: (df) pandas dataframe
    """
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data

import pandas as pd
import numpy as np
from stockstats import StockDataFrame as Sdf

# Sample stock data
data = pd.DataFrame({
    'date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
             '2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
    'tic': ['AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL',
            'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT'],
    'open': [145.0, 147.0, 149.0, 151.0, 153.0,
             240.0, 242.0, 244.0, 246.0, 248.0],
    'high': [148.0, 150.0, 152.0, 154.0, 156.0,
             243.0, 245.0, 247.0, 249.0, 251.0],
    'low': [144.0, 146.0, 148.0, 150.0, 152.0,
            239.0, 241.0, 243.0, 245.0, 247.0],
    'close': [147.0, 149.0, 151.0, 153.0, 155.0,
              242.0, 244.0, 246.0, 248.0, 250.0],
    'volume': [1000000, 1100000, 1200000, 1300000, 1400000,
               800000, 850000, 900000, 950000, 1000000]
})

# Let's say we want to add these technical indicators
# tech_indicator_list = ['macd', 'rsi_30', 'cci_30', 'dx_30']

print("Original data:")
print(data)

In [38]:
from finrl import config
from stockstats import StockDataFrame as sdf
class FeatureEngineer:
    """
    Provides methods for preprocessing stock price data.

    method:
    preprocess_data(): main method to feature engineering
    
    """
    def __init__(
        self,
        use_technical_indicator=True,
        tech_indicator_list=config.INDICATORS,
        use_vix=False,
        use_turbulence=False,
        user_defined_feature=False,
    ):
        self.use_technical_indicator = use_technical_indicator
        self.tech_indicator_list = tech_indicator_list
        self.use_vix = use_vix
        self.use_turbulence = use_turbulence
        self.user_defined_feature = user_defined_feature

    def preprocess_data(self, df):
        """Main method for feature engineering.
        @:param config: source dataframe
        @:return: a DataMatrices object
        """
        df = self.clean_data(df)

        if self.use_technical_indicator:
            df = self.add_technical_indicator(df)
            print("Successfully added technical indicators from FinRl library configuration!\n")
            print(f"Indicator list: {self.tech_indicator_list}")

        if self.use_vix:
            df=self.add_vix(df)
            print("Successfully added vix!")
            
        if self.use_turbulence:
            df = self.add_turbulence(df)
            print("Successfully added turbulence index!")
            
        if self.user_defined_feature:
            df = self.add_user_defined_feature(df)
            print("Successfully added user defined features!")
            
        df = df.ffill().bfill()
        return df

    def clean_data(self, data):
        """
        Data sorting, index transformation, pivot table creation, data cleaning, filtering.
        
        Any stock that has any missing values on any date will be completely dropped from the dataset.
        """
        df = data.copy()

        # sort the data frame by chronological date first, and by ticker in alphebatical order, change index after sorting. 
        df = df.sort_values(["date", "tic"], ignore_index=True)

        # set index of this data frame equivalent to represent unique dates
        # codes, unique_values = df.date.factorize()
        df.index = df.date.factorize()[0]
        # print("codes: ", codes)
        # print("unique values: ", unique_values)

        # create a pivot table from original one
        merge_close_df = df.pivot_table(index="date", columns="tic", values="close")

        # observe there are missing values in 'TSLA' on '2003-01-02' and '2003-01-03'
        merge_close_df = merge_close_df.dropna(axis=1) #drop the entire column
        tics = merge_close_df.columns
        
        # this filters the tic column by keeping the rows exist in 'tics' variable
        df = df[df.tic.isin(tics)] 
        
        return df

    # from stockstats import StockDataFrame as sdf
    def add_technical_indicator(self, data):
        """
        Transform basic OHLCV data 
        into a comprehensive dataset with technical analysis indicators 
        that help RL agents make better trading decisions.
        
        param: data is pd.DataFrame
        return: df 
        """
        df = data.copy()
        df = df.sort_values(by=["tic", "date"])

        # converts to a copy of df into StockDataFrame
        stock = sdf.retype(df.copy()) # it's still the same df, but can calculate technical indicators
        unique_ticker = stock.tic.unique()

        for indicator in self.tech_indicator_list:
            # print("indicators in the list: \n", self.tech_indicator_list)
            indicator_df = pd.DataFrame()
            
            for i, ticker in enumerate(unique_ticker):
                # note: ticker value is within the stock dataframe tic column
                stock_data = stock[stock.tic == ticker]
                
                # print(f"Stock data for {ticker}: \n", stock_data['close'])
                # print(f"Entire stock data: \n", stock_data)
                
                try:
                    # print(f"Current indicator: {indicator} for ticker: {ticker}\n")

                    # this creates a new series with a single column in stock_data sdf called temp_indicator, turned into pd.DataFrame
                    temp_indicator = stock_data[indicator] # this triggers sdf calculation based on this specific stock OHLCV value
                    temp_indicator = pd.DataFrame(temp_indicator) # Series -> df
                    
                    # print(f"\nCalculated {indicator} for {ticker}:")
                    # print(temp_indicator)
                    
                    # add columns tic, date to temp_indicator, for context
                    temp_indicator["tic"] = ticker 
                    temp_indicator["date"] = df[df.tic == ticker]["date"].to_list()
                    
                    # print(f"\nwith metadata added: ")
                    # print(temp_indicator)

                    #concatenate the modified temp_indicator df to indicator_df
                    # axis=0, stack vertically
                    # done accumulatively from each stock df information stored in temp_indicator
                    indicator_df = pd.concat(
                        [indicator_df, temp_indicator], axis=0, ignore_index=True
                    )
                    # print("Indicator df: \n", indicator_df)
                except Exception as e:
                    print(e)
                    
            # merge the df based on "tic", "date", effecively add indicator value with context to the original df
            df = df.merge(
                indicator_df[["tic","date", indicator]], on=["tic", "date"], how="left"
            )
            # print("After merge:\n", df)
            
            df = df.sort_values(by=["date", "tic"])
            
        return df
        

### use_technical_indicator test case
feaureEngineer_techindicator = FeatureEngineer(
    use_technical_indicator=True,
    use_turbulence=True,
    user_defined_feature=False)

feaureEngineer_techindicator.add_technical_indicator(data)

In [41]:
fe = FeatureEngineer(
    use_technical_indicator=True,
    use_turbulence=False,
    user_defined_feature=False)

# df = fe.preprocess_data(data)
df = fe.preprocess_data(df)

print(len(df.tic.unique()))
print(df)

Successfully added technical indicators from FinRl library configuration!

Indicator list: ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
28
             date       close        high         low        open      volume  \
0      2008-01-02    5.855758    6.018652    5.786934    5.988898  1079178800   
1      2008-01-02   32.223785   32.528046   31.988676   32.223785     7934400   
2      2008-01-02   38.898682   39.874196   38.708151   39.698909     8053700   
3      2008-01-02   63.481640   64.375743   63.027255   64.177869     4303000   
4      2008-01-02   44.507305   45.792806   44.141823   45.723490     6337800   
...           ...         ...         ...         ...         ...         ...   
96371  2021-09-01  146.831177  147.706925  145.844812  146.969448     1133800   
96372  2021-09-01  392.533875  394.904022  386.787168  391.772040     2034400   
96373  2021-09-01   43.478058   43.644249   43.367266   43.525541    12647300   
9637

import pandas as pd
import numpy as np

# Original messy data
data = pd.DataFrame({
    'date': ['2023-01-03', '2023-01-01', '2023-01-02', '2023-01-03', 
             '2023-01-01', '2023-01-02', '2023-01-01', '2023-01-03'],
    'tic': ['AAPL', 'AAPL', 'AAPL', 'MSFT', 'MSFT', 'MSFT', 'TSLA', 'TSLA'],
    'close': [150.0, 145.0, 148.0, 250.0, 245.0, 248.0, 800.0, np.nan]
})

print("Original data:")
print(data)

data.head()

In [ ]:
df.head()

### Add covariance matrix as states

In [42]:
# add covariance matrix as states

# returns the first tuple
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback = 252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback: i, :]
    price_lookback= data_lookback.pivot_table(index= 'date', columns= 'tic', values='close')
    return_lookback price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)

df_cov = pd.DataFrame({'date': df.date.unique()[lookback:], 
                       'cov_list':cov_list, 
                       'return_list': return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date', 'tic']).reset_index(drop=True)

SyntaxError: invalid syntax (626628075.py, line 14)

In [44]:
import pandas as pd
import numpy as np

# Simulate 3 years of daily data for 3 stocks (2021-2023)
dates = pd.date_range('2021-01-01', '2023-12-31', freq='D')
dates = [d for d in dates if d.weekday() < 5]  # Business days only

# Create sample data
data_rows = []
tickers = ['AAPL', 'MSFT', 'TSLA']

for date in dates:
    for ticker in tickers:
        # Simulate price data
        base_price = {'AAPL': 150, 'MSFT': 250, 'TSLA': 800}[ticker]
        noise = np.random.normal(0, 0.02)  # 2% daily volatility
        price = base_price * (1 + noise)
        
        data_rows.append({
            'date': date,
            'tic': ticker, 
            'close': price
        })

df = pd.DataFrame(data_rows)
print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.date.min()} to {df.date.max()}")
print(f"Unique dates: {len(df.date.unique())}")
print(f"Unique tickers: {df.tic.unique()}")

Dataset shape: (2343, 3)
Date range: 2021-01-01 00:00:00 to 2023-12-29 00:00:00
Unique dates: 781
Unique tickers: ['AAPL' 'MSFT' 'TSLA']
